In [47]:
from plot_helpers import *
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats 
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
from matplotlib.lines import Line2D
from scipy.stats import linregress
import matplotlib as mpl
%matplotlib qt

# Set global font sizes - for manuscript
mpl.rcParams['font.size'] = 20
mpl.rcParams['axes.labelsize'] = 28
mpl.rcParams['xtick.labelsize'] = 24
mpl.rcParams['ytick.labelsize'] = 24
mpl.rcParams['legend.fontsize'] = 18



In [46]:
# Load the statistics CSV file
combined_statistics_path = r"C:\Users\Feifei\Box\BR_remote_sensing\ebi_combined_statistics.csv"
df = pd.read_csv(combined_statistics_path)

# other File paths
root_dir = r'C:\Users\Feifei\Box\BR_remote_sensing\ebi_results'


In [48]:
# Initialize x-axis range (full range for the plot)
x_range = (60, 2e5)

# Group classifications: combine wandering types
wandering_types = ['HSW', 'LSW']
df['Group'] = df['Classification'].replace(wandering_types, 'Wandering')

# Define markers and colors for the groups
markers = {'Wandering': 's', 'B': '^'}
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Create the figure and axes
fig, ax = plt.subplots(figsize=(7, 6))

# Dictionary to store slope and R² information for the legend
legend_info = {}

# Loop through each group to compute and plot the regression line.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['mean_ebi_site']
    
    # Ensure x_data are positive (needed for log transformation)
    if any(x_data <= 0):
        raise ValueError(f"Group {group} contains non-positive x values, cannot use log transformation.")
    
    # Fit the regression model using mode "log-x"
    slope, intercept, predict = fit_regression(x_data, y_data, mode="log-x")
    
    # Compute R² on the original data:
    y_pred_data = predict(x_data)
    ss_res = np.sum((y_data - y_pred_data) ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r2 = 1 - ss_res / ss_tot
    
    # Generate regression line values over the full x_range.
    x_line = np.linspace(x_range[0], x_range[1], 100)
    y_line = predict(x_line)
    
    # Plot the regression line.
    ax.plot(x_line, y_line, color=colors[group], lw=1.5) # 3 for presentation
    
    # Store slope and R² for the legend.
    legend_info[group] = f"slope = {slope:.2f}, R² = {r2:.2f}"

# Plot data points with error bars for each group.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['mean_ebi_site']
    n = len(subset)
    y_error = subset['std_ebi_site'] / np.sqrt(n)
    
    # Plot error bars.
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=1,capthick=0.3, elinewidth=1)
    
    # Plot scatter points.
    ax.scatter(x_data, y_data, label=group, marker=markers[group],
               c=colors[group], edgecolors='black', linewidths=0.5, s=250, alpha=1, zorder=2) # linewidths=1 for presentation

# Set x-axis to log scale and adjust limits.
ax.set_xscale('log')
ax.set_xlim(*x_range)
ax.set_ylim(0.5, 10)

# Set labels and tick parameters.
ax.set_xlabel(r'${Q_{mean}}$(m³/s)')
ax.set_ylabel(r'${eBI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')

# Create custom legend elements showing the slope and R².
legend_elements = [
    Line2D([0], [0], marker=markers['Wandering'], color='w',
           label=f"Wandering: {legend_info['Wandering']}",
           markerfacecolor=colors['Wandering'], markersize=15, markeredgecolor='black', markeredgewidth=0.5), #edgewith=1 for presentation
    Line2D([0], [0], marker=markers['B'], color='w',
           label=f"Braided: {legend_info['B']}",
           markerfacecolor=colors['B'], markersize=15, markeredgecolor='black', markeredgewidth=0.5)
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)


plt.tight_layout()
plt.show()

fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\4_mean_eBI_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)



In [ ]:
# Find the data point with the maximum Mean_BI_annual
max_ebi_row = df.loc[df['mean_ebi_site'].idxmax()]
max_dim_Q = max_ebi_row['dim_Q']
max_ebi = max_ebi_row['mean_ebi_site']

# Define target rivers
target_rivers = ['Magdalena_Calamar', 'Yukon_Eagle']
discharge_data = {}
ebi_data = {}


# Load discharge and eBI data for each target river
for river in target_rivers:
    discharge_data[river] = load_discharge_data(river)
    ebi_annual_file = os.path.join(root_dir, river, 'rivgraph', 'eBI_results_annual.csv')
    if os.path.exists(ebi_annual_file):
        ebi_df = pd.read_csv(ebi_annual_file)
        ebi_data[river] = ebi_df[['Year', 'eBI']].groupby('Year').mean().reset_index()
    else:
        print(f"eBI results not found for {river}.")

# Merge discharge and eBI data for each target river
target_river_data = {}
for river in target_rivers:
    if river in discharge_data and river in ebi_data:
        merged_df = pd.merge(discharge_data[river], ebi_data[river], on='Year', how='inner')
        if not merged_df.empty:
            wetted_area = df.loc[df['River'] == river, 'wetted_area_avg_subannual'].values[0]
            if wetted_area > 0:
                # Calculate yearly dim_Q for target rivers
                g=9.81
                merged_df['dim_Q'] = merged_df['Q'] / (g**0.5 * wetted_area**(5/2))
                target_river_data[river] = merged_df
            else:
                print(f"No valid wetted_area value for {river}.")
        else:
            print(f"No common years for merging data in {river}.")
    else:
        print(f"Discharge or eBI data missing for {river}.")

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))

# Define markers and colors for wandering and braided rivers
markers = {'Wandering': 's', 'B': '^'}  # Combined HSW and LSW symbols
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Planform type descriptions
planform_type = {'Wandering': 'Wandering (HSW & LSW)', 'B': 'Braided'}

# Filter the DataFrame for rivers with Qm greater or equal to 17000 - Latrubesse 
high_Qm_df = df[df['Qm'] >= 17000]

if not high_Qm_df.empty:
    # Compute the min and max dim_Q for these rivers
    min_dim_Q_high = high_Qm_df['dim_Q'].min()
    max_dim_Q_high = high_Qm_df['dim_Q'].max()
    
    # Shade the region between min_dim_Q and max_dim_Q on the plot
    ax.axvspan(min_dim_Q_high, max_dim_Q_high, facecolor='orange', alpha=0.3, label='Qm ≥ 17000 range')
    ax.axvline(max_dim_Q, color='gray', linestyle='--', linewidth=2, label=f'Max eBI Mean at Q*={max_dim_Q:.2f}')
else:
    print("No rivers with Qm ≥ 17000 found.")

# Loop through classifications and plot wandering and braided rivers
for cls, description in [('B', 'Braided'), ('Wandering', 'Wandering (HSW & LSW)')]:
    if cls == 'Wandering':
        subset = df[df['Classification'].isin(['HSW', 'LSW'])]  # Combine HSW and LSW
    else:
        subset = df[df['Classification'] == cls]

    x_data = subset['dim_Q']
    y_data = subset['mean_ebi_site']

    n = len(subset)
    y_error = subset['std_ebi_site'] / np.sqrt(n)
    
    # Plot error bars.
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=1,capthick=0.3, elinewidth=1)
    ax.scatter(
        x_data, y_data, label=description, 
        marker=markers[cls], c=colors[cls], edgecolors='black', linewidths=0.5, s=250, alpha=1
    )

# Add target rivers to the plot
target_colors = {'Magdalena_Calamar': 'gray', 'Yukon_Eagle': 'gray'}  # Colors for the target rivers
for river, data in target_river_data.items():
    ax.scatter(
        data['dim_Q'], data['eBI'],  # Plot yearly dim_Q vs. yearly mean eBI
        label=river.replace('_', ' '), 
        s=250, alpha=1, edgecolors='black', linewidths=0.5, c=target_colors[river], marker='o'
    )


# Set axis labels and properties
ax.set_xlabel(r'${Q^*}$')  
ax.set_ylabel(r'${eBI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')
ax.set_xscale('log')
ax.set_ylim(0.5, 10)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\4_eBI_dim_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# (assume df is already defined)

# Combine wandering types
wandering_types = ['HSW', 'LSW']
df['Group'] = df['Classification'].replace(wandering_types, 'Wandering')

# x‐axis range
x_range = (df['dim_Q'].min() * 0.6, df['dim_Q'].max() * 1.5)

fig, ax = plt.subplots(figsize=(7, 6))
markers = {'Wandering': 's', 'B': '^'}
colors  = {'Wandering': '#FF6666', 'B': '#4169E1'}

for cls, label in [('B', 'Braided'), ('Wandering', 'Wandering')]:
    if cls == 'Wandering':
        subset = df[df['Group'] == 'Wandering']
    else:
        subset = df[df['Classification'] == cls]

    x = subset['dim_Q']
    y = subset['mean_ebi_site']
    yerr = subset['std_ebi_site'] / np.sqrt(len(subset))

    ax.errorbar(x, y, yerr=yerr, fmt='none', ecolor='black',
                capsize=6, elinewidth=1, alpha=1, zorder=1)
    ax.scatter(x, y,
               marker=markers[cls],
               c=colors[cls],
               edgecolors='black',
               linewidths=0.5,
               s=150,
               alpha=1,
               label=label)

# FIXED annotation loop:
for _, row in df.iterrows():
    river_val = row['River']
    if pd.isna(river_val):
        continue
    river_label = str(river_val).replace('_', ' ')
    ax.text(
        row['dim_Q'],
        row['mean_ebi_site'],
        river_label,
        fontsize=7,
        ha='right',
        va='bottom',
        clip_on=True
    )

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(*x_range)
ax.set_ylim(df['mean_ebi_site'].min() * 0.8, df['mean_ebi_site'].max() * 1.2)

ax.set_xlabel(r'${Q^*}$')
ax.set_ylabel(r'${eBI_{mean}}$')
ax.legend(frameon=False)

plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 1) Climate mapping
climate_full_names = {
    'A': 'Tropical',
    'B': 'Arid',
    'C': 'Temperate',
    'D': 'Cold',
    'E': 'Polar'
}
# drop Polar from the colours since there are none
climate_colors_full = {
    'Tropical':  '#1b9e77',
    'Arid':       '#d95f02',
    'Temperate':  '#7570b3',
    'Cold':       '#e7298a'
}

df['Climate Group'] = df['Climate Zone'].str[0]
df['Climate Group Full Name'] = df['Climate Group'].map(climate_full_names)

# only keep rows for which we have a colour
df_plot = df[df['Climate Group Full Name'].isin(climate_colors_full)].copy()

# 2) Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# shade the Qm ≥ 17000 region (no legend)
high_Qm = df_plot[df_plot['Qm'] >= 17000]
if not high_Qm.empty:
    ax.axvspan(
        high_Qm['dim_Q'].min(),
        high_Qm['dim_Q'].max(),
        facecolor='orange', alpha=0.3,
        label='_nolegend_'
    )
    ax.axvline(
        df_plot.loc[df_plot['eBI_BI_ratio_site'].idxmax(), 'dim_Q'],
        color='gray', linestyle='--', linewidth=2,
        label='_nolegend_'
    )

marker_size = 100  # adjust this smaller or larger as you like

for cls, marker in [('B', '^'), ('Wandering', 's')]:
    if cls == 'Wandering':
        sel = df_plot[df_plot['Classification'].isin(['HSW', 'LSW'])]
    else:
        sel = df_plot[df_plot['Classification'] == cls]

    x = sel['dim_Q']
    y = sel['eBI_BI_ratio_site']
    yerr = sel['std_ebi_site'] / np.sqrt(len(sel))
    colours = sel['Climate Group Full Name'].map(climate_colors_full)

    # # error bars behind points
    # ax.errorbar(
    #     x, y, yerr=yerr,
    #     fmt='none', ecolor='black',
    #     capsize=5, capthick=0.5, elinewidth=0.8,
    #     zorder=0, label='_nolegend_'
    # )
    ax.scatter(
        x, y,
        c=colours, marker=marker,
        edgecolors='black', linewidths=0.5,
        s=marker_size, zorder=1,
        label='_nolegend_'
    )

# axes styling
ax.set_xscale('log')
#ax.set_ylim(0.5, 10)
ax.set_xlabel(r'${Q^*}$')
ax.set_ylabel(r'${eBI/BI}$')

# legend for climate zones only
handles = [
    mpatches.Patch(color=c, label=zone)
    for zone, c in climate_colors_full.items()
]
ax.legend(
    handles=handles,
    title='Climate Zone',
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)

plt.tight_layout()
plt.show()



In [ ]:
stats = (
    df2
    .groupby('Climate Group Full Name')['dim_Q']
    .agg(['count', 'median', 'mean', 'std', 
          lambda s: np.percentile(s, 25), 
          lambda s: np.percentile(s, 75)])
    .rename(columns={'<lambda_0>':'Q25','<lambda_1>':'Q75'})
)
print(stats)


#### supporting information

In [ ]:
# Dynamically calculate x_range based on data (with some padding)
x_range = (df['Qbf point'].min() * 0.6, df['Qbf point'].max() * 1.3)



# Group classifications: combine wandering types
wandering_types = ['HSW', 'LSW']
df['Group'] = df['Classification'].replace(wandering_types, 'Wandering')

# Define markers and colors for the groups
markers = {'Wandering': 's', 'B': '^'}
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Create the figure and axes
fig, ax = plt.subplots(figsize=(7, 6))

# Dictionary to store slope and R² information for the legend
legend_info = {}

# Loop through each group to compute and plot the regression line.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qbf point']
    y_data = subset['mean_ebi_site']
    
    # Ensure x_data are positive (needed for log transformation)
    if any(x_data <= 0):
        raise ValueError(f"Group {group} contains non-positive x values, cannot use log transformation.")
    
    # Fit the regression model using mode "log-x"
    slope, intercept, predict = fit_regression(x_data, y_data, mode="log-x")
    
    # Compute R² on the original data:
    y_pred_data = predict(x_data)
    ss_res = np.sum((y_data - y_pred_data) ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r2 = 1 - ss_res / ss_tot
    
    # Generate regression line values over the full x_range.
    x_line = np.linspace(x_range[0], x_range[1], 100)
    y_line = predict(x_line)
    
    # Plot the regression line.
    ax.plot(x_line, y_line, color=colors[group], lw=1.5) # 3 for presentation
    
    # Store slope and R² for the legend.
    legend_info[group] = f"slope = {slope:.2f}, R² = {r2:.2f}"

# Plot data points with error bars for each group.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qbf point']
    y_data = subset['mean_ebi_site']
    n = len(subset)
    y_error = subset['std_bi_site'] / np.sqrt(n)
    
    # Plot error bars.
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=1,capthick=0.3, elinewidth=1)
    
    # Plot scatter points.
    ax.scatter(x_data, y_data, label=group, marker=markers[group],
               c=colors[group], edgecolors='black', linewidths=0.5, s=250, alpha=1, zorder=2) # linewidths=1 for presentation

# Set x-axis to log scale and adjust limits.
ax.set_xscale('log')
ax.set_xlim(*x_range)
ax.set_ylim(0.5, 10)

# Set labels and tick parameters.
ax.set_xlabel(r'${Q_{bf}}$(m³/s)')
ax.set_ylabel(r'${eBI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')

# Create custom legend elements showing the slope and R².
legend_elements = [
    Line2D([0], [0], marker=markers['Wandering'], color='w',
           label=f"Wandering: {legend_info['Wandering']}",
           markerfacecolor=colors['Wandering'], markersize=15, markeredgecolor='black', markeredgewidth=0.5), #edgewith=1 for presentation
    Line2D([0], [0], marker=markers['B'], color='w',
           label=f"Braided: {legend_info['B']}",
           markerfacecolor=colors['B'], markersize=15, markeredgecolor='black', markeredgewidth=0.5)
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)


plt.tight_layout()
plt.show()

fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\1_supp_bankful_eBI_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)

In [ ]:
# Initialize x-axis range (full range for the plot)
x_range = (60, 2e5)

# Group classifications: combine wandering types
wandering_types = ['HSW', 'LSW']
df['Group'] = df['Classification'].replace(wandering_types, 'Wandering')

# Define markers and colors for the groups
markers = {'Wandering': 's', 'B': '^'}
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Create the figure and axes
fig, ax = plt.subplots(figsize=(7, 6))

# Dictionary to store slope and R² information for the legend
legend_info = {}

# Loop through each group to compute and plot the regression line.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['mean_bi_site']
    
    # Ensure x_data are positive (needed for log transformation)
    if any(x_data <= 0):
        raise ValueError(f"Group {group} contains non-positive x values, cannot use log transformation.")
    
    # Fit the regression model using mode "log-x"
    slope, intercept, predict = fit_regression(x_data, y_data, mode="log-x")
    
    # Compute R² on the original data:
    y_pred_data = predict(x_data)
    ss_res = np.sum((y_data - y_pred_data) ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r2 = 1 - ss_res / ss_tot
    
    # Generate regression line values over the full x_range.
    x_line = np.linspace(x_range[0], x_range[1], 100)
    y_line = predict(x_line)
    
    # Plot the regression line.
    ax.plot(x_line, y_line, color=colors[group], lw=1.5) # 3 for presentation
    
    # Store slope and R² for the legend.
    legend_info[group] = f"slope = {slope:.2f}, R² = {r2:.2f}"

# Plot data points with error bars for each group.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['mean_bi_site']
    n = len(subset)
    y_error = subset['std_bi_site'] / np.sqrt(n)
    
    # Plot error bars.
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=1,capthick=0.3, elinewidth=1)
    
    # Plot scatter points.
    ax.scatter(x_data, y_data, label=group, marker=markers[group],
               c=colors[group], edgecolors='black', linewidths=0.5, s=250, alpha=1, zorder=2) # linewidths=1 for presentation

# Set x-axis to log scale and adjust limits.
ax.set_xscale('log')
ax.set_xlim(*x_range)
ax.set_ylim(0.5, 12.5)

# Set labels and tick parameters.
ax.set_xlabel(r'${Q_{mean}}$(m³/s)')
ax.set_ylabel(r'${BI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')

# Create custom legend elements showing the slope and R².
legend_elements = [
    Line2D([0], [0], marker=markers['Wandering'], color='w',
           label=f"Wandering: {legend_info['Wandering']}",
           markerfacecolor=colors['Wandering'], markersize=15, markeredgecolor='black', markeredgewidth=0.5), #edgewith=1 for presentation
    Line2D([0], [0], marker=markers['B'], color='w',
           label=f"Braided: {legend_info['B']}",
           markerfacecolor=colors['B'], markersize=15, markeredgecolor='black', markeredgewidth=0.5)
]
ax.legend(handles=legend_elements, loc='upper right', frameon=False)


plt.tight_layout()
plt.show()

fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\1_supp_mean_BI_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)

In [ ]:
# Find the data point with the maximum Mean_BI_annual
max_ebi_row = df.loc[df['mean_bi_site'].idxmax()]
max_dim_Q = max_ebi_row['dim_Q']
max_ebi = max_ebi_row['mean_bi_site']

# Define target rivers
target_rivers = ['Magdalena_Calamar', 'Yukon_Eagle']
discharge_data = {}
ebi_data = {}


# Load discharge and eBI data for each target river
for river in target_rivers:
    discharge_data[river] = load_discharge_data(river)
    ebi_annual_file = os.path.join(root_dir, river, 'rivgraph', 'BI_results_annual.csv')
    if os.path.exists(ebi_annual_file):
        ebi_df = pd.read_csv(ebi_annual_file)
        ebi_data[river] = ebi_df[['Year', 'BI']].groupby('Year').mean().reset_index()
    else:
        print(f"eBI results not found for {river}.")

# Merge discharge and eBI data for each target river
target_river_data = {}
for river in target_rivers:
    if river in discharge_data and river in ebi_data:
        merged_df = pd.merge(discharge_data[river], ebi_data[river], on='Year', how='inner')
        if not merged_df.empty:
            wetted_area = df.loc[df['River'] == river, 'wetted_area_avg_subannual'].values[0]
            if wetted_area > 0:
                # Calculate yearly dim_Q for target rivers
                g=9.81
                merged_df['dim_Q'] = merged_df['Q'] / (g**0.5 * wetted_area**(5/2))
                target_river_data[river] = merged_df
            else:
                print(f"No valid wetted_area value for {river}.")
        else:
            print(f"No common years for merging data in {river}.")
    else:
        print(f"Discharge or eBI data missing for {river}.")

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))

# Define markers and colors for wandering and braided rivers
markers = {'Wandering': 's', 'B': '^'}  # Combined HSW and LSW symbols
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Planform type descriptions
planform_type = {'Wandering': 'Wandering (HSW & LSW)', 'B': 'Braided'}

# Filter the DataFrame for rivers with Qm greater or equal to 17000 - Latrubesse 
high_Qm_df = df[df['Qm'] >= 17000]

if not high_Qm_df.empty:
    # Compute the min and max dim_Q for these rivers
    min_dim_Q_high = high_Qm_df['dim_Q'].min()
    max_dim_Q_high = high_Qm_df['dim_Q'].max()
    
    # Shade the region between min_dim_Q and max_dim_Q on the plot
    ax.axvspan(min_dim_Q_high, max_dim_Q_high, facecolor='orange', alpha=0.3, label='Qm ≥ 17000 range')
    ax.axvline(max_dim_Q, color='gray', linestyle='--', linewidth=2, label=f'Max eBI Mean at Q*={max_dim_Q:.2f}')
else:
    print("No rivers with Qm ≥ 17000 found.")

# Loop through classifications and plot wandering and braided rivers
for cls, description in [('B', 'Braided'), ('Wandering', 'Wandering (HSW & LSW)')]:
    if cls == 'Wandering':
        subset = df[df['Classification'].isin(['HSW', 'LSW'])]  # Combine HSW and LSW
    else:
        subset = df[df['Classification'] == cls]

    x_data = subset['dim_Q']
    y_data = subset['mean_bi_site']

    n = len(subset)
    y_error = subset['std_bi_site'] / np.sqrt(n)
    
    # Plot error bars.
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=1,capthick=0.3, elinewidth=1)
    ax.scatter(
        x_data, y_data, label=description, 
        marker=markers[cls], c=colors[cls], edgecolors='black', linewidths=0.5, s=250, alpha=1
    )

# Add target rivers to the plot
target_colors = {'Magdalena_Calamar': 'gray', 'Yukon_Eagle': 'gray'}  # Colors for the target rivers
for river, data in target_river_data.items():
    ax.scatter(
        data['dim_Q'], data['BI'],  # Plot yearly dim_Q vs. yearly mean eBI
        label=river.replace('_', ' '), 
        s=250, alpha=1, edgecolors='black', linewidths=0.5, c=target_colors[river], marker='o'
    )


# Set axis labels and properties
ax.set_xlabel(r'${Q^*}$')  
ax.set_ylabel(r'${BI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')
ax.set_xscale('log')
ax.set_ylim(0.5, 11)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


# Save the figure
fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\1_supp_BI_dim_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)

In [ ]:
# Dynamically calculate x_range based on data (with padding)
x_range = (df['Qm'].min() * 0.6, df['Qm'].max() * 1.5)

# Define markers and colors
markers = {'Wandering': 's', 'B': '^'}  # Combined HSW and LSW symbols
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Combine wandering classifications into one group
wandering_types = ['HSW', 'LSW']
df['Group'] = df['Classification'].replace(wandering_types, 'Wandering')

# Create the figure and axes
fig, ax = plt.subplots(figsize=(7, 6))

# Dictionary to store slope and R² info for the legend
legend_info = {}

# Loop through each group to compute and plot the regression line.
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['wetted_area_avg_subannual']
    
    # Ensure x_data are positive (needed for log transformation)
    if any(x_data <= 0):
        raise ValueError(f"Group {group} contains non-positive x values, cannot use log transformation.")
    
    # Fit regression using the unified function with mode "log-x"
    slope, intercept, predict = fit_regression(x_data, y_data, mode="log-log")
    
    # Generate regression line values over the full x_range
    x_line = np.linspace(x_range[0], x_range[1], 300)
    y_line = predict(x_line)
    
    # Plot the regression line
    ax.plot(x_line, y_line, color=colors[group], lw=1.5)
    
    # Compute R² on the original data
    y_pred = predict(x_data)
    ss_res = ((y_data - y_pred) ** 2).sum()
    ss_tot = ((y_data - y_data.mean()) ** 2).sum()
    r2 = 1 - ss_res / ss_tot
    
    # Store slope and R² information for the legend
    legend_info[group] = f"slope = {slope:.2f}, R² = {r2:.2f}"

# Plot data points with error bars
for group in ['Wandering', 'B']:
    subset = df[df['Group'] == group]
    x_data = subset['Qm']
    y_data = subset['wetted_area_avg_subannual']    
    ax.scatter(x_data, y_data, label=group, marker=markers[group],
               c=colors[group], edgecolors='black', s=250, alpha=1, zorder=2, linewidths=0.5)

# Set x-axis to log scale and adjust limits
ax.set_xscale('log')
ax.set_yscale('log')


ax.set_xlim(*x_range)
ax.set_ylim(bottom=-100)

# Set labels and tick parameters
ax.set_xlabel(r'${Q_{mean}}$(m³/s)')
ax.set_ylabel(r'$B$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')

# Create a legend with slope and R² information
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker=markers['Wandering'], color='w',
           label=f"Wandering: {legend_info['Wandering']}",
           markerfacecolor=colors['Wandering'], markersize=15, markeredgecolor='black', markeredgewidth=0.5),
    Line2D([0], [0], marker=markers['B'], color='w',
           label=f"Braided: {legend_info['B']}",
           markerfacecolor=colors['B'], markersize=15, markeredgecolor='black', markeredgewidth=0.5)
]
ax.legend(handles=legend_elements,loc='upper right', frameon=False)

plt.tight_layout()
plt.show()


fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\1_supp_wetted_area_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)

In [ ]:
# Compute new dim_Q for all rivers in df using the Qbf point value
g = 9.81  # gravitational constant
df['dim_Q'] = df['Qm'] / (g**0.5 * df['wetted_area_avg_subannual']**(5/2))

# Find the data point with the maximum Mean_BI_annual
max_ebi_row = df.loc[df['mean_bi_site'].idxmax()]
max_dim_Q = max_ebi_row['dim_Q']
max_ebi = max_ebi_row['mean_bi_site']

# Define target rivers
target_rivers = ['Magdalena_Calamar', 'Yukon_Eagle']
discharge_data = {}
ebi_data = {}

# Load discharge and eBI data for each target river
for river in target_rivers:
    discharge_data[river] = load_discharge_data(river)
    ebi_annual_file = os.path.join(root_dir, river, 'rivgraph', 'BI_results_annual.csv')
    if os.path.exists(ebi_annual_file):
        ebi_df = pd.read_csv(ebi_annual_file)
        ebi_data[river] = ebi_df[['Year', 'BI']].groupby('Year').mean().reset_index()
    else:
        print(f"eBI results not found for {river}.")

# Merge discharge and eBI data for each target river
target_river_data = {}
for river in target_rivers:
    if river in discharge_data and river in ebi_data:
        merged_df = pd.merge(discharge_data[river], ebi_data[river], on='Year', how='inner')
        if not merged_df.empty:
            wetted_area = df.loc[df['River'] == river, 'wetted_area_avg_subannual'].values[0]
            if wetted_area > 0:
                # Calculate yearly dim_Q for target rivers (formula kept the same)
                g = 9.81
                merged_df['dim_Q'] = merged_df['Q'] / (g**0.5 * wetted_area**(5/2))
                target_river_data[river] = merged_df
            else:
                print(f"No valid wetted_area value for {river}.")
        else:
            print(f"No common years for merging data in {river}.")
    else:
        print(f"Discharge or eBI data missing for {river}.")

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))

# Define markers and colors for wandering and braided rivers
markers = {'Wandering': 's', 'B': '^'}  # Combined HSW and LSW symbols
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Planform type descriptions
planform_type = {'Wandering': 'Wandering (HSW & LSW)', 'B': 'Braided'}

# Filter the DataFrame for rivers with Qm greater or equal to 17000 - Latrubesse 
high_Qm_df = df[df['Qm'] >= 17000]

if not high_Qm_df.empty:
    # Compute the min and max dim_Q for these rivers
    min_dim_Q_high = high_Qm_df['dim_Q'].min()
    max_dim_Q_high = high_Qm_df['dim_Q'].max()
    
    # Shade the region between min_dim_Q and max_dim_Q on the plot
    ax.axvspan(min_dim_Q_high, max_dim_Q_high, facecolor='orange', alpha=0.3, label='Qm ≥ 17000 range')
    ax.axvline(max_dim_Q, color='gray', linestyle='--', linewidth=2, label=f'Max eBI Mean at Q*={max_dim_Q:.2f}')

else:
    print("No rivers with Qm ≥ 17000 found.")

# Loop through classifications and plot wandering and braided rivers
for cls, description in [('Wandering', 'Wandering (HSW & LSW)'), ('B', 'Braided')]:
    if cls == 'Wandering':
        subset = df[df['Classification'].isin(['HSW', 'LSW'])]  # Combine HSW and LSW
        scatter_z = 2
        error_z = 1.5
    else:  # Braided
        subset = df[df['Classification'] == cls]
        scatter_z = 3  # higher zorder so that these points appear on top
        error_z = 2.5

    x_data = subset['dim_Q']
    y_data = subset['mean_bi_site']

    n = len(subset)
    y_error = subset['std_bi_site'] / np.sqrt(n)
    
    # Plot error bars with a slightly lower zorder than the points
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=error_z, capthick=0.3, elinewidth=1)
    ax.scatter(
        x_data, y_data, label=description, 
        marker=markers[cls], c=colors[cls], edgecolors='black', linewidths=0.5, s=250, alpha=1,
        zorder=scatter_z
    )


# Add target rivers to the plot with a high zorder
target_colors = {'Magdalena_Calamar': 'gray', 'Yukon_Eagle': 'gray'}  # Colors for the target rivers
for river, data in target_river_data.items():
    ax.scatter(
        data['dim_Q'], data['BI'],  # Plot yearly dim_Q vs. yearly mean eBI
        label=river.replace('_', ' '), 
        s=250, alpha=1, edgecolors='black', linewidths=0.5, c=target_colors[river],
        marker='o', zorder=10  # high zorder so that these points appear on top
    )


# Set axis labels and properties
ax.set_xlabel(r'${Q^*}$')  
ax.set_ylabel(r'${BI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')
ax.set_xscale('log')
ax.set_ylim(0.5, 11)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\2_supp_BI_dim_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)


In [ ]:
# Compute new dim_Q for all rivers in df using the Qbf point value
g = 9.81  # gravitational constant


# Find the data point with the maximum Mean_BI_annual
max_ebi_row = df.loc[df['mean_ebi_site'].idxmax()]
max_dim_Q = max_ebi_row['dim_Q']
max_ebi = max_ebi_row['mean_ebi_site']

# Define target rivers
target_rivers = ['Magdalena_Calamar', 'Yukon_Eagle']
discharge_data = {}
ebi_data = {}

# Load discharge and eBI data for each target river
for river in target_rivers:
    discharge_data[river] = load_discharge_data(river)
    ebi_annual_file = os.path.join(root_dir, river, 'rivgraph', 'eBI_results_annual.csv')
    if os.path.exists(ebi_annual_file):
        ebi_df = pd.read_csv(ebi_annual_file)
        ebi_data[river] = ebi_df[['Year', 'eBI']].groupby('Year').mean().reset_index()
    else:
        print(f"eBI results not found for {river}.")

# Merge discharge and eBI data for each target river
target_river_data = {}
for river in target_rivers:
    if river in discharge_data and river in ebi_data:
        merged_df = pd.merge(discharge_data[river], ebi_data[river], on='Year', how='inner')
        if not merged_df.empty:
            wetted_area = df.loc[df['River'] == river, 'wetted_area_avg_subannual'].values[0]
            if wetted_area > 0:
                # Calculate yearly dim_Q for target rivers (formula kept the same)
                g = 9.81
                merged_df['dim_Q'] = merged_df['Q'] / (g**0.5 * wetted_area**(5/2))
                target_river_data[river] = merged_df
            else:
                print(f"No valid wetted_area value for {river}.")
        else:
            print(f"No common years for merging data in {river}.")
    else:
        print(f"Discharge or eBI data missing for {river}.")

# Plotting
fig, ax = plt.subplots(figsize=(7, 6))

# Define markers and colors for wandering and braided rivers
markers = {'Wandering': 's', 'B': '^'}  # Combined HSW and LSW symbols
colors = {'Wandering': '#FF6666', 'B': '#4169E1'}

# Planform type descriptions
planform_type = {'Wandering': 'Wandering (HSW & LSW)', 'B': 'Braided'}

# Filter the DataFrame for rivers with Qm greater or equal to 17000 - Latrubesse 
high_Qm_df = df[df['Qm'] >= 17000]

if not high_Qm_df.empty:
    # Compute the min and max dim_Q for these rivers
    min_dim_Q_high = high_Qm_df['dim_Q'].min()
    max_dim_Q_high = high_Qm_df['dim_Q'].max()
    
    # Shade the region between min_dim_Q and max_dim_Q on the plot
    ax.axvspan(min_dim_Q_high, max_dim_Q_high, facecolor='orange', alpha=0.3, label='Qm ≥ 17000 range')
    ax.axvline(max_dim_Q, color='gray', linestyle='--', linewidth=2, label=f'Max eBI Mean at Q*={max_dim_Q:.2f}')

else:
    print("No rivers with Qm ≥ 17000 found.")

# Loop through classifications and plot wandering and braided rivers
for cls, description in [('Wandering', 'Wandering (HSW & LSW)'), ('B', 'Braided')]:
    if cls == 'Wandering':
        subset = df[df['Classification'].isin(['HSW', 'LSW'])]  # Combine HSW and LSW
        scatter_z = 2
        error_z = 1.5
    else:  # Braided
        subset = df[df['Classification'] == cls]
        scatter_z = 3  # higher zorder so that these points appear on top
        error_z = 2.5

    x_data = subset['dim_Q']
    y_data = subset['mean_ebi_site']

    n = len(subset)
    y_error = subset['std_ebi_site'] / np.sqrt(n)
    
    # Plot error bars with a slightly lower zorder than the points
    ax.errorbar(x_data, y_data, yerr=y_error, fmt='none', ecolor='black',
                capsize=8, alpha=1, zorder=error_z, capthick=0.3, elinewidth=1)
    ax.scatter(
        x_data, y_data, label=description, 
        marker=markers[cls], c=colors[cls], edgecolors='black', linewidths=0.5, s=250, alpha=1,
        zorder=scatter_z
    )


# Add target rivers to the plot with a high zorder
target_colors = {'Magdalena_Calamar': 'gray', 'Yukon_Eagle': 'gray'}  # Colors for the target rivers
for river, data in target_river_data.items():
    ax.scatter(
        data['dim_Q'], data['eBI'],  # Plot yearly dim_Q vs. yearly mean eBI
        label=river.replace('_', ' '), 
        s=250, alpha=1, edgecolors='black', linewidths=0.5, c=target_colors[river],
        marker='o', zorder=10  # high zorder so that these points appear on top
    )


# Set axis labels and properties
ax.set_xlabel(r'${Q^*}$')  
ax.set_ylabel(r'${eBI_{mean}}$')
ax.tick_params(axis='x')
ax.tick_params(axis='y')
ax.set_xscale('log')
ax.set_ylim(0.5, 11)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

print("x value of the gray line (Max eBI Mean Q*):", max_dim_Q)

# Save the figure
fig.savefig(r"C:\Users\Feifei\Box\BR_remote_sensing\figures\2_supp_eBI_dim_Q.pdf", 
            format='pdf', dpi=500, bbox_inches='tight', transparent=True)


In [ ]:
# ——————————————————————————————
# 1) user‐adjustable parameters (no longer use constant Nt)
# ——————————————————————————————
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

g   = 9.81        # gravity [m/s^2]
Fr  = 0.1
WDR = 20

# ——————————————————————————————
# 2) recompute Q* for the master dataframe, using mean_ebi_site as Nt
# ——————————————————————————————
# alpha is now a Series, with Nt = mean_ebi_site for each row
alpha_series = Fr / (df['mean_ebi_site'] * WDR) ** 1.5
df['dim_Q'] = df['Qm'] / (alpha_series * np.sqrt(g) * df['Width(m)'] ** 2.5)

# find the river with max mean BI for your vertical line, if still needed
max_ebi_row = df.loc[df['mean_ebi_site'].idxmax()]
max_dim_Q   = max_ebi_row['dim_Q']

# ——————————————————————————————
# 3) load & process target rivers, computing dim_Q using that river's mean_ebi_site
# ——————————————————————————————
target_rivers     = ['Magdalena_Calamar', 'Yukon_Eagle']
discharge_data    = {}
ebi_data          = {}
target_river_data = {}

for r in target_rivers:
    discharge_data[r] = load_discharge_data(r)
    bi_file = os.path.join(root_dir, r, 'rivgraph', 'eBI_results_annual.csv')
    if os.path.exists(bi_file):
        tmp = (
            pd.read_csv(bi_file)[['Year', 'eBI']]
            .groupby('Year')
            .mean()
            .reset_index()
        )
        ebi_data[r] = tmp

for r in target_rivers:
    if r in discharge_data and r in ebi_data:
        m = pd.merge(discharge_data[r], ebi_data[r], on='Year', how='inner')
        if not m.empty:
            # get mean_ebi_site for this river from the master df
            mean_ebi_vals = df.loc[df['River'] == r, 'mean_ebi_site'].values
            if len(mean_ebi_vals) and mean_ebi_vals[0] > 0:
                Nt_r = mean_ebi_vals[0]
                alpha_r = Fr / (Nt_r * WDR) ** 1.5
                # use wetted area (Lz) from the master df as before
                Lz_vals = df.loc[df['River'] == r, 'wetted_area_avg_subannual'].values
                if len(Lz_vals) and Lz_vals[0] > 0:
                    Lz = Lz_vals[0]
                    m['dim_Q'] = m['Q'] / (alpha_r * np.sqrt(g) * Lz ** 2.5)
                    target_river_data[r] = m

# ——————————————————————————————
# 4) plot everything (omit gray points)
# ——————————————————————————————
fig, ax = plt.subplots(figsize=(7, 6))

# plot by class
markers = {'B': '^', 'Wandering': 's'}
colors  = {'B': '#4169E1', 'Wandering': '#FF6666'}

for cls, desc in [('B', 'Braided'), ('Wandering', 'Wandering (HSW&LSW)')]:
    if cls == 'Wandering':
        sub = df[df['Classification'].isin(['HSW', 'LSW'])]
    else:
        sub = df[df['Classification'] == cls]

    x = sub['dim_Q']
    y = sub['mean_ebi_site']
    n = len(sub)
    yerr = sub['std_ebi_site'] / np.sqrt(n)
    ax.errorbar(
        x,
        y,
        yerr=yerr,
        fmt='none',
        ecolor='black',
        capsize=6,
        capthick=0.5,
        elinewidth=1
    )
    ax.scatter(
        x,
        y,
        label=desc,
        marker=markers[cls],
        c=colors[cls],
        edgecolors='black',
        linewidths=0.5,
        s=200
    )

# (Gray points removed; if you want to plot target rivers with a different style, insert your own code here.)

ax.set_xscale('log')
ax.set_ylim(0.5, 11)
ax.set_xlabel(r'$Q^*$')
ax.set_ylabel(r'$eBI_{\rm mean}$')
plt.tight_layout()
plt.show()

